In [1]:

import sys, os

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%run ../load_magic/storage.py
%run ../load_magic/dataframes.py
%pprint
%matplotlib inline
import matplotlib.pyplot as plt

# Use the following only if you are on a high definition device
from matplotlib_inline.backend_inline import set_matplotlib_formats
set_matplotlib_formats('retina')

import re
from math import cos, sin, pi, sqrt, atan
import numpy as np
from PIL import Image
from matplotlib.pyplot import imshow
from spiral_utils import StraussHoweUtilities

s = Storage()
shu = StraussHoweUtilities(s=s)
%who

Pretty printing has been turned OFF
FILEPATH_REGEX	 Image	 Path	 Storage	 StraussHoweUtilities	 URL_REGEX	 atan	 bs	 cos	 
csv	 example_iterrows	 get_column_descriptions	 get_max_rsquared_adj	 get_page_soup	 get_page_tables	 get_wiki_tables	 imshow	 io	 
math	 np	 os	 pd	 pi	 pickle	 plt	 random	 re	 
s	 set_matplotlib_formats	 shu	 sin	 sm	 sns	 sqrt	 stats	 sys	 
urllib	 urlretrieve	 


In [3]:

cw_history_year_dict = s.load_object('cw_history_year_dict')
dataframes_dict = s.load_dataframes(patriline_df='patriline_df', turnings_df='turnings_df')

Attempting to load C:\Users\daveb\OneDrive\Documents\GitHub\Strauss-Howe\saves\pkl\patriline_df.pkl.
Attempting to load C:\Users\daveb\OneDrive\Documents\GitHub\Strauss-Howe\saves\pkl\turnings_df.pkl.


In [4]:

index_name = 'Patriarch Name'
patriline_df = dataframes_dict['patriline_df']
if 'Unnamed: 0' in patriline_df.columns:
    patriline_df = patriline_df.set_index('Unnamed: 0')
elif index_name in patriline_df.columns:
    patriline_df = patriline_df.set_index(index_name)
patriline_df.index.name = index_name

In [5]:

index_name = 'turning_name'
turnings_df = dataframes_dict['turnings_df']
if 'Unnamed: 0' in turnings_df.columns:
    turnings_df = turnings_df.set_index('Unnamed: 0')
elif index_name in turnings_df.columns:
    turnings_df = turnings_df.set_index(index_name)
turnings_df.index.name = index_name

In [6]:

py_file_header_str = '''
import bpy

# Get spiral data
patriarch_coords_dict = {}
'''

In [7]:

py_file_footer_str = """

for patriarch_name, coords in patriarch_coords_dict.items():
    
    # create the Curve Datablock
    curveData = bpy.data.curves.new(patriarch_name, type='CURVE')
    curveData.dimensions = '3D'
    curveData.resolution_u = 2
    
    # map coords to spline
    polyline = curveData.splines.new('NURBS')
    polyline.points.add(len(coords))
    for i, coord in enumerate(coords):
        x,y,z = coord
        polyline.points[i].co = (x, y, z, 1)

    # create Object
    curveOB = bpy.data.objects.new(patriarch_name, curveData)

    # attach to scene and validate context
    scn = bpy.context.scene
    scn.objects.link(curveOB)
    scn.objects.active = curveOB
    curveOB.select = True"""

In [8]:

py_bezier_file_footer_str = """

for patriarch_name, coords in patriarch_coords_dict.items():
    
    # create the Curve Datablock
    curveData = bpy.data.curves.new(patriarch_name, type='CURVE')
    curveData.dimensions = '3D'
    curveData.resolution_u = 2
    
    # map coords to spline
    polyline = curveData.splines.new('BEZIER')
    coords_count = len(coords)
    polyline.bezier_points.add(coords_count-1)
    for i, coord in enumerate(coords):
        x, y, z = coord
        polyline.bezier_points[i].co = (x, y, z)
    for i in range(coords_count):
        if (i == 0) or (i == coords_count-1):
            polyline.bezier_points[i].handle_left = polyline.bezier_points[i].handle_right = polyline.bezier_points[i].co
        else:
            x, y, z = coords[i-1]
            polyline.bezier_points[i].handle_left = (x, y, z)
            x, y, z = coords[i+1]
            polyline.bezier_points[i].handle_right = (x, y, z)

    # create Object
    curveOB = bpy.data.objects.new(patriarch_name, curveData)

    # attach to scene and validate context
    scn = bpy.context.scene
    scn.objects.link(curveOB)
    scn.objects.active = curveOB
    curveOB.select = True"""

In [10]:

stopped_year = 2030
py_dir = os.path.join('..', 'py')
os.makedirs(name=py_dir, exist_ok=True)
out_file_path = os.path.join(py_dir, f'output_{stopped_year}.py')
shu.save_stopped_babbitt_plot_as_blender_script(history_year_dict=cw_history_year_dict, stopped_year=stopped_year, out_file_path=out_file_path,
                              footer_str=py_bezier_file_footer_str, py_file_header_str=py_file_header_str, verbose=True)

Saving to C:\Users\daveb\OneDrive\Documents\GitHub\Strauss-Howe\py\output_2030.py


In [ ]:

min_year = patriline_df['Year of Birth'].min()
for stopped_year in range(min_year, 2031):
    out_file_path = os.path.join(py_dir, 'output_{}.py'.format(stopped_year))
    shu.save_stopped_babbitt_plot_as_blender_script(cw_history_year_dict, stopped_year, out_file_path, py_file_footer_str)

In [ ]:
%%javascript
var kernel = IPython.notebook.kernel;
var body = document.body;
var attribs = body.attributes;
var command = "notebook_path = " + "'"+attribs['data-notebook-path'].value+"'";
kernel.execute(command);

In [ ]:
# %load ../../load_magic/nbviewer.py

from IPython.display import HTML

notebook_viewer_url = 'https://nbviewer.jupyter.org/github/dbabbitt/notebooks/blob/master/'
notebook_viewer_url += '/'.join(notebook_path.split('/')[1:])
html_str = 'Click <a href="{}" target="_blank">here</a> to view notebook in nbviewer.'
HTML(html_str.format(notebook_viewer_url))